In [1]:
import csv
import pandas as pd
import numpy as np
import math
import scipy
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [2]:
appVolDat = pd.read_csv('filingVolumes.csv')
prediction = appVolDat.iloc[1405]
appVolDat = appVolDat.iloc[1320:1404]
appVolDat = appVolDat.reset_index(drop=True)
maximum = max(appVolDat.volume)
appVolDat['vol'] = appVolDat.volume/maximum

trend = pd.read_csv('trend_CostOfPatent.csv').trends.iloc[72:156].reset_index(drop=True)/100
appVolDat['trend1'] = trend#(trend1 - np.mean(trend1)) / np.std(trend1)

trend = pd.read_csv('trend_utilityVDesign.csv').trends.iloc[72:156].reset_index(drop=True)/100
appVolDat['trend2'] = trend#(trend1 - np.mean(trend1)) / np.std(trend1)

trend = pd.read_csv('trend_uspto.csv').trends.iloc[72:156].reset_index(drop=True)/100
appVolDat['trend3'] = trend#(trend1 - np.mean(trend1)) / np.std(trend1)

appVolDat

,month,volume,vol,trend1,trend2,trend3
0,1/1/2010,31365,0.377523,0.52,0.00,0.57
1,2/1/2010,33237,0.400055,0.36,0.29,0.60
2,3/1/2010,43055,0.518229,0.57,0.00,0.66
3,4/1/2010,38781,0.466785,0.30,0.00,0.65
4,5/1/2010,36019,0.433541,0.44,0.00,0.66
5,6/1/2010,42086,0.506566,0.59,0.60,0.77
6,7/1/2010,36667,0.441340,0.31,0.25,0.65
7,8/1/2010,37406,0.450235,0.33,0.00,0.65
8,9/1/2010,40013,0.481614,0.22,0.00,0.60
9,10/1/2010,38245,0.460334,0.21,0.23,0.67


In [3]:
for i in range(1,13):
    appVolDat['volShift'+str(i)] = appVolDat.vol.shift(i)
appVolDat = appVolDat.iloc[12:]
appVolDat

,month,volume,vol,trend1,trend2,trend3,volShift1,volShift2,volShift3,volShift4,volShift5,volShift6,volShift7,volShift8,volShift9,volShift10,volShift11,volShift12
12,1/1/2011,35259,0.424393,0.53,0.00,0.66,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541,0.466785,0.518229,0.400055,0.377523
13,2/1/2011,35057,0.421962,0.64,0.19,0.68,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541,0.466785,0.518229,0.400055
14,3/1/2011,45603,0.548898,0.37,0.17,0.67,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541,0.466785,0.518229
15,4/1/2011,38163,0.459347,0.35,0.36,0.72,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541,0.466785
16,5/1/2011,38934,0.468627,0.43,0.00,0.61,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541
17,6/1/2011,43581,0.524560,0.53,0.00,0.64,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566
18,7/1/2011,35761,0.430435,0.43,0.35,0.60,0.524560,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340
19,8/1/2011,41170,0.495540,0.51,0.16,0.65,0.430435,0.524560,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235
20,9/1/2011,47684,0.573946,0.35,0.33,0.64,0.495540,0.430435,0.524560,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614
21,10/1/2011,37863,0.455736,0.60,0.16,0.63,0.573946,0.495540,0.430435,0.524560,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334


In [4]:
def func(X,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o):
    return a*(X['volShift1']) + b*(X['volShift2']) + c*(X['volShift3']) + \
d*(X['volShift4']) + e*(X['volShift5']) + f*(X['volShift6']) + \
g*(X['volShift7']) + h*(X['volShift8']) + i*(X['volShift9']) + \
j*(X['volShift10']) + k*(X['volShift11']) + l*(X['volShift12']) + \
m*(X['trend1']) + n*(X['trend2'])# + o*(X['trend3'])

In [5]:
y = np.array(appVolDat['vol'])
fitParams, fitCovariances = curve_fit(func, appVolDat, y)
print(fitParams)

[ 0.19257485  0.20704434  0.274455    0.08659749  0.1002589   0.1504424
  0.07515833 -0.10246435  0.07953762 -0.22009279 -0.11989832  0.33739444
 -0.07367586 -0.00528278  1.        ]


C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [6]:
ypred = func(appVolDat, fitParams[0], fitParams[1], fitParams[2], fitParams[3], fitParams[4], fitParams[5], fitParams[6], 
            fitParams[7], fitParams[8], fitParams[9], fitParams[10], fitParams[11], fitParams[12], fitParams[13], fitParams[14])

In [7]:
appVolDat.insert(3, 'volPred', ypred)
print(ypred)

12    0.436151
13    0.413606
14    0.500488
15    0.463290
16    0.455897
17    0.502150
18    0.463477
19    0.445727
20    0.507739
21    0.460441
22    0.468397
23    0.554646
24    0.448187
25    0.438898
26    0.519619
27    0.456978
28    0.479857
29    0.528176
30    0.474203
31    0.495138
32    0.550811
33    0.499947
34    0.506482
35    0.579455
36    0.491353
37    0.494038
38    0.535105
39    0.562116
40    0.586196
41    0.643281
        ...   
54    0.543555
55    0.538976
56    0.576707
57    0.530477
58    0.490343
59    0.574834
60    0.464379
61    0.457983
62    0.582841
63    0.453609
64    0.484274
65    0.524902
66    0.495315
67    0.444421
68    0.527115
69    0.473937
70    0.453188
71    0.526866
72    0.439894
73    0.369065
74    0.412689
75    0.343293
76    0.323970
77    0.356419
78    0.299738
79    0.296719
80    0.350239
81    0.296184
82    0.295037
83    0.290575
Length: 72, dtype: float64


In [8]:
appVolDat['vol'] = appVolDat['vol'] * maximum
appVolDat['volPred'] = appVolDat['volPred'] * maximum
appVolDat

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,month,volume,vol,volPred,trend1,trend2,trend3,volShift1,volShift2,volShift3,volShift4,volShift5,volShift6,volShift7,volShift8,volShift9,volShift10,volShift11,volShift12
12,1/1/2011,35259,35259.0,36235.839167,0.53,0.00,0.66,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541,0.466785,0.518229,0.400055,0.377523
13,2/1/2011,35057,35057.0,34362.794731,0.64,0.19,0.68,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541,0.466785,0.518229,0.400055
14,3/1/2011,45603,45603.0,41581.035121,0.37,0.17,0.67,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541,0.466785,0.518229
15,4/1/2011,38163,38163.0,38490.571374,0.35,0.36,0.72,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541,0.466785
16,5/1/2011,38934,38934.0,37876.407671,0.43,0.00,0.61,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566,0.433541
17,6/1/2011,43581,43581.0,41719.160696,0.53,0.00,0.64,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340,0.506566
18,7/1/2011,35761,35761.0,38506.102448,0.43,0.35,0.60,0.524560,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235,0.441340
19,8/1/2011,41170,41170.0,37031.461814,0.51,0.16,0.65,0.430435,0.524560,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614,0.450235
20,9/1/2011,47684,47684.0,42183.429886,0.35,0.33,0.64,0.495540,0.430435,0.524560,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334,0.481614
21,10/1/2011,37863,37863.0,38253.898083,0.60,0.16,0.63,0.573946,0.495540,0.430435,0.524560,0.468627,0.459347,0.548898,0.421962,0.424393,0.542025,0.459359,0.460334


In [9]:
print("Training Error: ", np.sum(np.absolute(appVolDat['volPred']-appVolDat['vol'])) / len(appVolDat))

Training Error:  3165.065057651867
